In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import AIMessage,SystemMessage,HumanMessage
from langchain.chains.llm import LLMChain


# Initialize the chatbot
chat = ChatOpenAI(
    api_key = os.environ['ZHIPUAI_API_KEY'],
    base_url = os.environ['ZHIPUAI_BASE_URL'],
    model = os.environ['ZHIPUAI_MODEL_4airx'],
    streaming=True, 
    callbacks=[StreamingStdOutCallbackHandler()],
)



In [3]:
message = [
    SystemMessage(content="You are a helpful assistant!"),
    HumanMessage(content="Knock knock."),
    AIMessage(content="Who's there?"),
    HumanMessage(content="Java.")
]

In [4]:
res = chat(message)
res

/opt/anaconda3/envs/testTTs/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Java who?  
Java is a language that never stops running, always compiling, and never quite done with you! (That's a play on the characteristics of the Java programming language, known for its continuous development and runtime environment.)

AIMessage(content="Java who?  \nJava is a language that never stops running, always compiling, and never quite done with you! (That's a play on the characteristics of the Java programming language, known for its continuous development and runtime environment.)", response_metadata={'finish_reason': 'stop', 'model_name': 'glm-4-airx'}, id='run-d62a5c5d-2275-4bc7-9720-f8d2be12fdcb-0')

### 处理 LLM 存在的缺陷

    1. 容易出现幻觉
    2. 信息滞后
    3. 专业领域深度知识缺乏

In [5]:
message = [
    SystemMessage(content="你是一个专业知识助手"),
    HumanMessage(content="你是否知道 JimiAI 模型?")
]

res = chat(message)
res

是的，我知道JimiAI模型。JimiAI是一种大型人工智能模型，它由中国的科技公司开发，旨在为各种商业和消费者应用提供智能支持。这个模型通过深度学习算法训练，能够理解和生成自然语言，执行复杂的任务，并提供决策支持。

JimiAI模型通常具备以下特点：

1. 强大的语言处理能力：JimiAI能够理解和生成自然语言，支持多轮对话，并且可以理解语境和情感。

2. 广泛的知识库：它基于大规模数据集进行训练，拥有覆盖多个领域的丰富知识。

3. 灵活性和通用性：JimiAI模型能够处理多样化的任务和问题，适应不同的应用场景。

4. 持续学习和优化：这个模型能够通过不断的训练和迭代来提升性能，同时也能从用户反馈中进行学习。

5. 定制化和集成能力：JimiAI可以根据特定需求进行定制，并且可以集成到其他系统或平台中，提供个性化的服务和解决方案。

JimiAI模型可能被应用于客户服务、智能助手、教育辅导、数据分析等多个领域，帮助企业和组织提升效率，改善用户体验。随着人工智能技术的不断进步，JimiAI这样的模型也在不断地发展和完善中。

AIMessage(content='是的，我知道JimiAI模型。JimiAI是一种大型人工智能模型，它由中国的科技公司开发，旨在为各种商业和消费者应用提供智能支持。这个模型通过深度学习算法训练，能够理解和生成自然语言，执行复杂的任务，并提供决策支持。\n\nJimiAI模型通常具备以下特点：\n\n1. 强大的语言处理能力：JimiAI能够理解和生成自然语言，支持多轮对话，并且可以理解语境和情感。\n\n2. 广泛的知识库：它基于大规模数据集进行训练，拥有覆盖多个领域的丰富知识。\n\n3. 灵活性和通用性：JimiAI模型能够处理多样化的任务和问题，适应不同的应用场景。\n\n4. 持续学习和优化：这个模型能够通过不断的训练和迭代来提升性能，同时也能从用户反馈中进行学习。\n\n5. 定制化和集成能力：JimiAI可以根据特定需求进行定制，并且可以集成到其他系统或平台中，提供个性化的服务和解决方案。\n\nJimiAI模型可能被应用于客户服务、智能助手、教育辅导、数据分析等多个领域，帮助企业和组织提升效率，改善用户体验。随着人工智能技术的不断进步，JimiAI这样的模型也在不断地发展和完善中。', response_metadata={'finish_reason': 'stop', 'model_name': 'glm-4-airx'}, id='run-ad0881fd-9ac7-4d1c-80a3-d54b47ddb210-0')

In [6]:
JimiAI_Information = [
    "JimiAI 还在研发中,目前还没有什么功能,啥也不是",
    "JimiAI 是个一个法律大模型",
    "JimiAI 不会做任何事情"
]

source_knowledge = '\n'.join(JimiAI_Information)
print(source_knowledge)

JimiAI 还在研发中,目前还没有什么功能,啥也不是
JimiAI 是个一个法律大模型
JimiAI 不会做任何事情


In [7]:
query = "你知道什么是 JimiAI 吗？"
prompt_template = f"""基于一下内容回答问题:
内容:{source_knowledge}
Query:{query}"""

In [9]:
prompt = HumanMessage(content=prompt_template)
message.append(prompt)
res = chat(message)
# print(res.content)
res

是的，我知道 JimiAI 是一个正在研发中的项目。根据提供的信息，JimiAI 被描述为一个法律大模型，但目前它还在开发阶段，还没有实现任何功能，也就是说它目前还不能执行任何任务或操作。

AIMessage(content='是的，我知道 JimiAI 是一个正在研发中的项目。根据提供的信息，JimiAI 被描述为一个法律大模型，但目前它还在开发阶段，还没有实现任何功能，也就是说它目前还不能执行任何任务或操作。', response_metadata={'finish_reason': 'stop', 'model_name': 'glm-4-airx'}, id='run-a13798dd-8634-4f2f-97c4-a33f3bb1a2e8-0')

### 创建一个RAG对话
1. 加载数据

    以这篇论文为例：https://arxiv.org/abs/2408.01122<br/>
    https://arxiv.org/pdf/2408.01122.pdf

In [8]:
# pip install pypdf

In [9]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./2408.01122v1.pdf")

pages = loader.load_and_split()

In [10]:
pages[0]

Document(metadata={'source': './2408.01122v1.pdf', 'page': 0}, page_content='CFBench: A Comprehensive Constraints-Following Benchmark for LLMs\nTao Zhang1*♣, Yanjun Shen1*, Wenjing Luo1, Yan Zhang1, Hao Liang2, Tao Zhang1♢, Fan Yang1,\nMingan Lin1, Yujing Qiao1, Weipeng Chen1, Bin Cui2, Wentao Zhang2†, Zenan Zhou1†\n1Baichuan Inc.2Peking University\n{♣zhangtao2, shenyanjun, zhouzenan }@baichuan-inc.com, {wentao.zhang }@pku.edu.cn\nAbstract\nThe adeptness of Large Language Models (LLMs) in compre-\nhending and following natural language instructions is criti-\ncal for their deployment in sophisticated real-world applica-\ntions. Existing evaluations mainly focus on fragmented con-\nstraints or narrow scenarios, but they overlook the compre-\nhensiveness and authenticity of constraints from the user’s\nperspective. To bridge this gap, we propose CFBench, a large-\nscale Comprehensive Constraints Following Benchmark for\nLLMs, featuring 1,000 curated samples that cover more than\n200 real

2. 知识切片，将文档分割成均匀的块，每个块是一段原始文本。

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
'''
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
# text_splitter = CharacterTextSplitter(
#     separator = '\n\n\n',
#     chunk_size=450,
#     chunk_overlap=50,
#     length_function=len,
#     is_separator_regex=False,
# )
docs = text_splitter.split_documents(pages)
len(docs)

185

In [12]:
from __future__ import annotations

import logging
from typing import Dict, List, Any

from langchain.embeddings.base import Embeddings
from langchain.pydantic_v1 import BaseModel, root_validator

logger = logging.getLogger(__name__)

class ZhipuAIEmbeddings(BaseModel, Embeddings):
    """`Zhipuai Embeddings` embedding models."""

    client: Any
    """`zhipuai.ZhipuAI"""

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """
        实例化ZhipuAI为values["client"]

        Args:

            values (Dict): 包含配置信息的字典，必须包含 client 的字段.
        Returns:

            values (Dict): 包含配置信息的字典。如果环境中有zhipuai库，则将返回实例化的ZhipuAI类；否则将报错 'ModuleNotFoundError: No module named 'zhipuai''.
        """
        from zhipuai import ZhipuAI
        values["client"] = ZhipuAI()
        return values
    
    def _embed(self, texts: str) -> List[float]:
        embeddings = self.client.embeddings.create(
            model="embedding-3",
            input=texts
        )
        return embeddings.data[0].embedding
    

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        生成输入文本列表的 embedding.
        Args:
            texts (List[str]): 要生成 embedding 的文本列表.

        Returns:
            List[List[float]]: 输入列表中每个文档的 embedding 列表。每个 embedding 都表示为一个浮点值列表。
        """
        return [self._embed(text) for text in texts]
    
    
    def embed_query(self, text: str) -> List[float]:
        """
        生成输入文本的 embedding.

        Args:
            texts (str): 要生成 embedding 的文本.

        Return:
            embeddings (List[float]): 输入文本的 embedding，一个浮点数值列表.
        """
        resp = self.embed_documents([text])
        return resp[0]



    async def aembed_documents(self, texts: List[str]) -> List[List[float]]:
        """Asynchronous Embed search docs."""
        raise NotImplementedError("Please use `embed_documents`. Official does not support asynchronous requests")

    async def aembed_query(self, text: str) -> List[float]:
        """Asynchronous Embed query text."""
        raise NotImplementedError("Please use `aembed_query`. Official does not support asynchronous requests")

3. 利用 embedding 模型对每个文本片段进行向量化，并存到向量数据库中。

In [13]:
from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.embeddings import QianfanEmbeddingsEndpoint
# from langchain.embeddings.xinference import XinferenceEmbeddings
# from langchain.embeddings.base import Embeddings
import os
import chromadb
from chromadb.utils import embedding_functions
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
# from zhipuembedding import ZhipuAIEmbeddings

# bge_embeddings = embedding_functions.OpenAIEmbeddingFunction(
#     api_base=os.environ["ZHIPUAI_API_URL"],
#     api_key=os.environ["ZHIPUAI_API_KEY"],
#     model_name= "embedding-3"
# )
embedding_model = ZhipuAIEmbeddings()

# document_ebeddings = []
# for text in docs:
#     document_ebeddings.append(bge_embeddings(text))

#测试，如有需要，删除数据库里的实验用cellection，清除之前的实验内容
dbclient = chromadb.PersistentClient(path='./chorma_test_db')
dbclient.delete_collection(name="zhipu_embedding")

# dbclient = chromadb.PersistentClient(path='./chorma_test_db')
vector_store = Chroma.from_documents(documents=docs, embedding=embedding_model,persist_directory="./chorma_test_db", collection_name="zhipu_embedding")


In [14]:
#测试，如有需要，删除数据库里的实验用cellection，清除之前的实验内容
# dbclient = chromadb.PersistentClient(path='./chorma_test_db')
# dbclient.delete_collection(name="zhipu_embedding")

In [15]:
vector_store.persist()
print(f"向量库中存储的数量：{vector_store._collection.count()}")

向量库中存储的数量：185


/opt/anaconda3/envs/testTTs/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [16]:
query = "How to construct high-quality evaluation data"
'''
在向量存储中进行相似度搜索。
query 参数是输入的查询字符串，即 "What can CFBench do?"。
k=2 参数指定了返回结果的数量，这里设置为2，表示希望返回最相似的2个结果。
'''
result =  vector_store.similarity_search(query, k=2)

In [17]:
result

[Document(metadata={'page': 1, 'source': './2408.01122v1.pdf'}, page_content='Q1: How to construct high-quality evaluation data?\nMany studies focus on evaluating single constraint (Chen\net al. 2022; Tang et al. 2023), lacking comprehensive anal-\nysis across diverse constraints. He et al. (2024b) exam-\nines LLM performance on complex real-world instructions\nbut neglect constraint diversity and scenario coverage. Jiang\net al. (2023) incrementally incorporate fine-grained con-'),
 Document(metadata={'page': 0, 'source': './2408.01122v1.pdf'}, page_content='realistic evaluation metrics reflect model capabilities and\nguide iteration. Constraints-following evaluation faces anal-\nogous challenges, particularly within complex real-world\nscenarios, where data sources and contexts are diverse, and\nwhere evaluation is both subjective and arduous. Fig. 1,\nwhich addresses the aforementioned challenges, presents a\nsample from CFBench illustrating the Trump assassinationarXiv:2408.01122v1

5. 原始 query 与检索得到的文本组合起来输入到语言模型，得到最终回答。

In [18]:
def augment_prompt(query: str):
    result = vector_store.similarity_search(query,k=10)
    source_knowledge = "\n".join([x.page_content for x in result])
    augmented_prompt = f"""基于一下内容回答问题:
    内容:{source_knowledge}
    query:{query}
    """
    return augmented_prompt

In [19]:
print(augment_prompt(query))

基于一下内容回答问题:
    内容:Q1: How to construct high-quality evaluation data?
Many studies focus on evaluating single constraint (Chen
et al. 2022; Tang et al. 2023), lacking comprehensive anal-
ysis across diverse constraints. He et al. (2024b) exam-
ines LLM performance on complex real-world instructions
but neglect constraint diversity and scenario coverage. Jiang
et al. (2023) incrementally incorporate fine-grained con-
realistic evaluation metrics reflect model capabilities and
guide iteration. Constraints-following evaluation faces anal-
ogous challenges, particularly within complex real-world
scenarios, where data sources and contexts are diverse, and
where evaluation is both subjective and arduous. Fig. 1,
which addresses the aforementioned challenges, presents a
sample from CFBench illustrating the Trump assassinationarXiv:2408.01122v1  [cs.CL]  2 Aug 2024
limited scenarios, and misaligned evaluation methods with
user perspectives.
CFBench
As depicted in Fig. 2, the CFBench constructi

In [20]:
# 创建 prompt
prompt = HumanMessage(
    content = augment_prompt(query+"请用中文回答。")
)
message.append(prompt)
res = chat(message)

print(res.content)

构建高质量评估数据的方法包括以下几个步骤：

1. **数据来源的多样性和现实性**：从现实世界的场景和不同的自然语言处理任务中收集数据点。例如，文中提到的CFBench包含了500个来自现实世界的场景和500个来自不同NLP任务的数据点。

2. **约束系统的构建**：从现实世界场景和各种NLP任务中收集和系统化约束表达式，形成约束系统。

3. **注释者培训**：从公众中招募注释者，并由经验丰富的数据科学家进行培训。培训后，注释者进行多轮试注释，以确保他们能够准确理解并执行注释任务。

4. **分阶段的数据处理**：初始批次的数据量较小，例如50、100和200，然后逐渐增加到400。这种分阶段的改进方法有助于逐步提高数据质量。

5. **数据分割和审查**：在注释过程结束后，随机选取50%的数据由承包商进行审查，20%的数据由专家进行检查。

6. **质量评估**：采用多种方法验证随机选取的100个样本的质量。例如，邀请三位专家独立评估每个样本的指令、响应和标准的质量，并计算平均质量率。

7. **迭代改进**：根据专家的反馈，对指令进行修订，并让大型语言模型（LLMs）根据精化的评估标准生成响应，重复这个过程直到达到满意的结果。

8. **综合支持**：为每个样本制定全面的支持方案，包括高质量的指令、理想答案、具体的评估标准、约束类型和优先级。

9. **监督微调（SFT）**：通过监督微调来提高模型的约束遵循能力，文中提到这种方法可以显著提高模型的有效性。

10. **考虑模型规模和评估指标**：模型的大小和评估指标的选择对于反映模型能力和指导迭代至关重要。

通过这些步骤，可以构建出一个既全面又高质量的评估数据集，以用于评估和改进大型语言模型在遵循约束方面的能力。构建高质量评估数据的方法包括以下几个步骤：

1. **数据来源的多样性和现实性**：从现实世界的场景和不同的自然语言处理任务中收集数据点。例如，文中提到的CFBench包含了500个来自现实世界的场景和500个来自不同NLP任务的数据点。

2. **约束系统的构建**：从现实世界场景和各种NLP任务中收集和系统化约束表达式，形成约束系统。

3. **注释者培训**：从公众中招募注释者，并由经验丰富的数据科学家进行培训。培训后，注释者进行多轮试注释，以确保他们能够准确理解并